In [14]:
import pandas as pd

data = pd.read_csv('/root/global_terrorism.csv', encoding='ISO-8859-1')

print(data.head())


<ipython-input-14-a73800a248eb>:3: DtypeWarning: Columns (4,6,31,33,61,62,63,76,79,90,92,94,96,114,115,121) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('/root/global_terrorism.csv', encoding='ISO-8859-1')


        eventid  iyear  imonth  iday approxdate  extended resolution  country  \
0  197000000001   1970       7     2        NaN         0        NaN       58   
1  197000000002   1970       0     0        NaN         0        NaN      130   
2  197001000001   1970       1     0        NaN         0        NaN      160   
3  197001000002   1970       1     0        NaN         0        NaN       78   
4  197001000003   1970       1     0        NaN         0        NaN      101   

          country_txt  region  ... addnotes scite1 scite2  scite3  dbsource  \
0  Dominican Republic       2  ...      NaN    NaN    NaN     NaN      PGIS   
1              Mexico       1  ...      NaN    NaN    NaN     NaN      PGIS   
2         Philippines       5  ...      NaN    NaN    NaN     NaN      PGIS   
3              Greece       8  ...      NaN    NaN    NaN     NaN      PGIS   
4               Japan       4  ...      NaN    NaN    NaN     NaN      PGIS   

   INT_LOG  INT_IDEO INT_MISC INT_ANY 

In [17]:

summary_data = data.groupby(['iyear', 'attacktype1_txt']).size().reset_index(name='counts')
print(summary_data.head())


   iyear                 attacktype1_txt  counts
0   1970                   Armed Assault      61
1   1970                   Assassination      22
2   1970               Bombing/Explosion     333
3   1970  Facility/Infrastructure Attack     174
4   1970                       Hijacking      11


In [19]:
from IPython.display import HTML
import json


data_json = json.dumps(summary_data.to_dict(orient='records'))


html_string = f"""
<div id="d3-plot"></div>
<script src="https://d3js.org/d3.v6.min.js"></script>
<script>
var data = {data_json};

// Set the dimensions and margins of the graph
var margin = {{top: 30, right: 30, bottom: 70, left: 60}},
    width = 960 - margin.left - margin.right,
    height = 500 - margin.top - margin.bottom;

// Append the svg object to the body of the page
var svg = d3.select("#d3-plot")
  .append("svg")
    .attr("width", width + margin.left + margin.right)
    .attr("height", height + margin.top + margin.bottom)
  .append("g")
    .attr("transform",
          "translate(" + margin.left + "," + margin.top + ")");

// X axis
var x = d3.scaleBand()
  .range([ 0, width ])
  .domain(data.map(function(d) {{ return d.iyear; }}))
  .padding(0.2);
svg.append("g")
  .attr("transform", "translate(0," + height + ")")
  .call(d3.axisBottom(x))
  .selectAll("text")
    .attr("transform", "translate(-10,0)rotate(-45)")
    .style("text-anchor", "end");

// Add Y axis
var y = d3.scaleLinear()
  .domain([0, d3.max(data, function(d) {{ return +d.counts; }})])
  .range([ height, 0]);
svg.append("g")
  .call(d3.axisLeft(y));

// Bars
svg.selectAll("mybar")
  .data(data)
  .enter()
  .append("rect")
    .attr("x", function(d) {{ return x(d.iyear); }})
    .attr("y", function(d) {{ return y(d.counts); }})
    .attr("width", x.bandwidth())
    .attr("height", function(d) {{ return height - y(d.counts); }})
    .attr("fill", "#69b3a2");

</script>
"""


HTML(html_string)


In [20]:
html_string = f"""
<div id="scatter-plot"></div>
<script src="https://d3js.org/d3.v6.min.js"></script>
<script>
var data = {data_json};

var margin = {{top: 20, right: 20, bottom: 30, left: 40}},
    width = 960 - margin.left - margin.right,
    height = 500 - margin.top - margin.bottom;

var x = d3.scaleLinear().range([0, width]);
var y = d3.scaleLinear().range([height, 0]);

var svg = d3.select("#scatter-plot").append("svg")
    .attr("width", width + margin.left + margin.right)
    .attr("height", height + margin.top + margin.bottom)
  .append("g")
    .attr("transform", "translate(" + margin.left + "," + margin.top + ")");

// Scale the range of the data
x.domain([d3.min(data, function(d) {{ return d.iyear; }}), d3.max(data, function(d) {{ return d.iyear; }})]);
y.domain([0, d3.max(data, function(d) {{ return d.nkill; }})]);

// Add the scatterplot
svg.selectAll("dot")
    .data(data)
  .enter().append("circle")
    .attr("r", 5)
    .attr("cx", function(d) {{ return x(d.iyear); }})
    .attr("cy", function(d) {{ return y(d.nkill); }})
    .style("fill", "blue");

// Add the X Axis
svg.append("g")
    .attr("transform", "translate(0," + height + ")")
    .call(d3.axisBottom(x));

// Add the Y Axis
svg.append("g")
    .call(d3.axisLeft(y));
</script>
"""

HTML(html_string)

In [21]:
html_string = """
<div id="radial-cluster"></div>
<script src="https://d3js.org/d3.v4.min.js"></script>
<script>
var width = 960,
    height = 800,
    radius = (Math.min(width, height) / 2) - 10;

var format = d3.format(",d");

var x = d3.scaleLinear()
    .range([0, 2 * Math.PI]);

var y = d3.scaleSqrt()
    .range([0, radius]);

var color = d3.scaleOrdinal(d3.schemeCategory20);

var partition = d3.partition();

var arc = d3.arc()
    .startAngle(function(d) {{ return Math.max(0, Math.min(2 * Math.PI, x(d.x0))); }})
    .endAngle(function(d) {{ return Math.max(0, Math.min(2 * Math.PI, x(d.x1))); }})
    .innerRadius(function(d) {{ return Math.max(0, y(d.y0)); }})
    .outerRadius(function(d) {{ return Math.max(0, y(d.y1)); }});

var svg = d3.select("#radial-cluster").append("svg")
    .style("width", "100%")
    .style("height", "auto")
    .attr("viewBox", "0 0 " + width + " " + height)
  .append("g")
    .attr("transform", "translate(" + width / 2 + "," + (height / 2) + ")");

// Assume data is already set up hierarchically
var root = d3.hierarchy(data)
    .sum(function(d) {{ return d.size; }})
    .sort(function(a, b) {{ return b.value - a.value; }});

partition(root);

svg.selectAll("path")
    .data(root.descendants())
  .enter().append("path")
    .attr("d", arc)
    .style("fill", function(d) {{ return color((d.children ? d : d.parent).data.name); }})
    .on("click", click);

function click(d) {
    svg.transition()
        .duration(750)
        .tween("scale", function() {
            var xd = d3.interpolate(x.domain(), [d.x0, d.x1]),
                yd = d3.interpolate(y.domain(), [d.y0, 1]),
                yr = d3.interpolate(y.range(), [d.y0 ? 20 : 0, radius]);
            return function(t) { x.domain(xd(t)); y.domain(yd(t)).range(yr(t)); };
        })
      .selectAll("path")
        .attrTween("d", function(d) { return function() { return arc(d); }; });
}

function computeTextRotation(d) {
    return (x((d.x0 + d.x1) / 2) - Math.PI / 2) / Math.PI * 180;
}
</script>
"""

HTML(html_string)

In [22]:
html_string = """
<div id="tree-map"></div>
<script src="https://d3js.org/d3.v4.min.js"></script>
<script>
var width = 960,
    height = 500;

var format = d3.format(",d");

var treemap = d3.treemap()
    .tile(d3.treemapResquarify)
    .size([width, height])
    .round(true)
    .paddingInner(1);

var svg = d3.select("#tree-map").append("svg")
    .attr("width", width)
    .attr("height", height)
    .style("font", "10px sans-serif");

var root = d3.hierarchy(data)
    .eachBefore(function(d) { d.data.id = (d.parent ? d.parent.data.id + "." : "") + d.data.name; })
    .sum(sumBySize)
    .sort(function(a, b) { return b.height - a.height || b.value - a.value; });

treemap(root);

var cell = svg.selectAll("g")
    .data(root.leaves())
  .enter().append("g")
    .attr("transform", function(d) { return "translate(" + d.x0 + "," + d.y0 + ")"; });

cell.append("rect")
    .attr("id", function(d) { return d.data.id; })
    .attr("width", function(d) { return d.x1 - d.x0; })
    .attr("height", function(d) { return d.y1 - d.y0; })
    .attr("fill", function(d) { return color(d.parent.data.id); });

cell.append("clipPath")
    .attr("id", function(d) { return "clip-" + d.data.id; })
  .append("use")
    .attr("xlink:href", function(d) { return "#" + d.data.id; });

cell.append("text")
    .attr("clip-path", function(d) { return "url(#clip-" + d.data.id + ")"; })
  .selectAll("tspan")
    .data(function(d) { return d.data.name.split(/(?=[A-Z][^A-Z])/g); })
  .enter().append("tspan")
    .attr("x", 4)
    .attr("y", function(d, i) { return 13 + i * 10; })
    .text(function(d) { return d; });

function sumBySize(d) {
  return d.size;
}
</script>
"""

HTML(html_string)